In [18]:
import cv2
#import keras_cv
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras import layers, Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [19]:
model = keras.Sequential([
    layers.Input((28,28,3)),
    layers.Conv2D(filters=32,kernel_size=(3,3),activation='relu',padding='valid'),
    layers.MaxPooling2D(pool_size=(3,3),strides=(1,1),padding='valid'),

    layers.Conv2D(filters=64,kernel_size=(3,3),activation='relu',padding='valid'),
    layers.MaxPooling2D(pool_size=(3,3),strides=(1,1),padding='valid'),

    layers.Flatten(),
    layers.Dense(64),
    layers.Dense(10,activation='softmax')
])

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)                    │ (None, 26, 26, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 24, 24, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 22, 22, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 20, 20, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 25600)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │       1,638,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 10)                  │             650 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,658,506 (6.33 MB)

 Trainable params: 1,658,506 (6.33 MB)

 Non-trainable params: 0 (0.00 B)

In [20]:
(trX, trY), (tsX, tsY) = mnist.load_data()

# normalization
trainX = trX.astype('float32') / 255.0
testX = tsX.astype('float32') / 255.0

n = 20000
m = 1000
b = 4

# Convert grayscale to RGB
trainX=np.stack([cv2.cvtColor(trainX[i],cv2.COLOR_GRAY2BGR) for i in range(n)])
testX=np.stack([cv2.cvtColor(testX[i],cv2.COLOR_GRAY2BGR) for i in range(m)])
#One Hot Encoding
trainY=to_categorical(trY[:n],num_classes=10)
testY=to_categorical(tsY[:m],num_classes=10)

In [21]:
#Augmentation
augdataGen=ImageDataGenerator(
    rotation_range=10,
    height_shift_range=.19,
    width_shift_range=.11,
    shear_range=4.3,
    zoom_range=.21
)

aug_train_data_frame=augdataGen.flow(trainX,trainY,batch_size=b,shuffle=True)
aug_test_data_frame=augdataGen.flow(testX,testY,batch_size=b,shuffle=True)

In [22]:
#Convert Aug Data to list
aug_trainX,aug_trainY=[],[]
aug_testX,aug_testY=[],[]

for i in range(n//b):
    aug_imgs,aug_labels=next(aug_train_data_frame)
    aug_trainX.extend(aug_imgs)
    aug_trainY.extend(aug_labels)

for i in range(m//b):
    aug_imgs,aug_labels=next(aug_test_data_frame)
    aug_testX.extend(aug_imgs)
    aug_testY.extend(aug_labels)

In [23]:
#Convert List to Numpy Array
aug_trainX=np.array(aug_trainX)
aug_trainY=np.array(aug_trainY)
aug_testX=np.array(aug_testX)
aug_testY=np.array(aug_testY)

In [24]:
#Original+Augmented
x_com=np.concatenate((trainX,aug_trainX))
y_com=np.concatenate((trainY,aug_trainY))

print(x_com.shape)
print(y_com.shape)

(40000, 28, 28, 3)
(40000, 10)


In [25]:
#Model Compile
opt=Adam(learning_rate=0.0005)
my_callback=[keras.callbacks.EarlyStopping(monitor='val_loss',patience=5)]
model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])

In [26]:
history = model.fit(trainX, trainY, validation_split=0.2, batch_size=4, shuffle=True, epochs=20, callbacks=my_callback)
plot_history(history)

Epoch 1/20
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 34s 8ms/step - accuracy: 0.8959 - loss: 0.3525 - val_accuracy: 0.9772 - val_loss: 0.0690
Epoch 2/20
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 33s 8ms/step - accuracy: 0.9779 - loss: 0.0708 - val_accuracy: 0.9793 - val_loss: 0.0629
Epoch 3/20
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 33s 8ms/step - accuracy: 0.9854 - loss: 0.0471 - val_accuracy: 0.9787 - val_loss: 0.0737
Epoch 4/20
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 33s 8ms/step - accuracy: 0.9897 - loss: 0.0340 - val_accuracy: 0.9822 - val_loss: 0.0554
Epoch 5/20
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 31s 8ms/step - accuracy: 0.9923 - loss: 0.0242 - val_accuracy: 0.9805 - val_loss: 0.0652
Epoch 6/20
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 31s 8ms/step - accuracy: 0.9910 - loss: 0.0245 - val_accuracy: 0.9772 - val_loss: 0.1022
Epoch 7/20
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 31s 8ms/step - accuracy: 0.9940 - loss: 0.0177 - val_accuracy: 0.9703 - val_loss: 0.1247
Epoch 8/20
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 32s 8ms/step - accuracy: 0.9934 - loss: 0

NameError: name 'plot_history' is not defined